Copyright (c) MONAI Consortium 
Licensed under the Apache License, Version 2.0 (the "License"); 
you may not use this file except in compliance with the License. 
You may obtain a copy of the License at 
    http://www.apache.org/licenses/LICENSE-2.0 
Unless required by applicable law or agreed to in writing, software 
distributed under the License is distributed on an "AS IS" BASIS, 
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 
See the License for the specific language governing permissions and 
limitations under the License. 

Created by Udbhav Ram - Cardenas Lab, University of Alabama at Birmingham for contribution towards the MONAI project 


# Datalist Generator

## Setup environment

In [22]:
!python3 -c "import monai" || pip install -q "monai"

## Setup imports

In [23]:
import os
import json
import random
import shutil
import tempfile
from monai.config import print_config
from monai.apps import download_and_extract
print_config()

MONAI version: 1.1.0
Numpy version: 1.18.5
Pytorch version: 1.13.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.17.2
Pillow version: 7.2.0
Tensorboard version: 2.3.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.50.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.7.2
pandas version: 1.1.2
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, p

# Setup paths to your data #

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

# Download sample MSD Dataset #

In [ ]:
msd_task = "Task04_Hippocampus"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"

compressed_file = os.path.join(root_dir, msd_task + ".tar")
dataroot = os.path.join(root_dir, msd_task)

if not os.path.exists(dataroot):
    download_and_extract(resource, compressed_file, root_dir)

# MSD dataset structure follows the following convention: #

In [ ]:
test_dir = os.path.join(dataroot, "imagesTs/")
train_dir = os.path.join(dataroot, "imagesTr/")
label_dir = os.path.join(dataroot, "labelsTr/")

# Construct skeleton JSON to populate with your own data #

In [ ]:
datalist_json = {
    "testing": [],
    "training": []
}

# Populate JSON with test data #

In [ ]:
for file in os.listdir(test_dir):
    datalist_json["testing"].append({
        "image": './imagesTs/' + file,
    })

# Visualise testing data #

In [ ]:
datalist_json['testing'][:10]

# Populate with training images and labels in your directory #

In [ ]:
for file in os.listdir(train_dir):
    datalist_json["training"].append({
        "image": './imagesTr/' + file,
        "label": './labelsTr/' + file,
        "fold": 0  # Initialize as single fold
    })

# Visualise training data #

In [ ]:
datalist_json['training'][:10]

# Split training data into 5 random folds #

In [ ]:
random.seed(42)

for i in range(5):
    for j in range(len(datalist_json["training"])):
        if random.random() < 0.2:
            datalist_json["training"][j]["fold"] = i

# Visualise final training data with all randomised folds #

In [ ]:
datalist_json['training'][:5]

# Save JSON to file #

In [ ]:
with open('datalist.json', 'w', encoding='utf-8') as f:
    json.dump(datalist_json, f, ensure_ascii=False, indent=4)

# Cleanup temporary files #

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)